## 2 Analyzing the Data

1. For the series in the “hedge fund series” tab, report the following summary statistics:
(a) mean
(b) volatility
(c) Sharpe ratio
Annualize these statistics

In [31]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
from janitor import clean_names, remove_empty
sns.set_theme(style='white', palette='cubehelix', font_scale = 0.8, rc={'figure.figsize': (8, 6)})

In [32]:
hedge_df = (pd.read_excel('/Users/victoriashi/Desktop/Quant-Fin-Algo-Trading/HW2/proshares_analysis_data.xlsx', sheet_name='hedge_fund_series', parse_dates=True, index_col=0).pipe(remove_empty)
)
hedge_df.head()

,HFRIFWI Index,MLEIFCTR Index,MLEIFCTX Index,HDG US Equity,QAI US Equity
0,-0.03,-0.03,-0.03,-0.03,-0.01
1,-0.04,-0.03,-0.03,-0.03,-0.02
2,0.03,0.04,0.04,0.05,0.03
3,-0.01,-0.01,-0.01,-0.03,-0.01
4,-0.00,0.00,0.00,0.01,0.00


In [33]:
hedge_df.index, hedge_df.columns, hedge_df.mean() * 12

(RangeIndex(start=0, stop=142, step=1),
 Index(['HFRIFWI Index', 'MLEIFCTR Index', 'MLEIFCTX Index', 'HDG US Equity',
        'QAI US Equity'],
       dtype='object'),
 HFRIFWI Index    0.04
 MLEIFCTR Index   0.03
 MLEIFCTX Index   0.03
 HDG US Equity    0.02
 QAI US Equity    0.02
 dtype: float64)

In [34]:
# Calculate the mean, volatility and Sharpe ratio
hedge_df_summary = pd.DataFrame(index=hedge_df.columns)
hedge_df_summary['Mean'] = hedge_df.mean() * 12
hedge_df_summary['Vol'] = hedge_df.std() * np.sqrt(12)
hedge_df_summary['Sharpe'] = (hedge_df.mean() / hedge_df.std()) * np.sqrt(12)
hedge_df_summary

,Mean,Vol,Sharpe
HFRIFWI Index,0.04,0.06,0.69
MLEIFCTR Index,0.03,0.06,0.54
MLEIFCTX Index,0.03,0.06,0.51
HDG US Equity,0.02,0.06,0.33
QAI US Equity,0.02,0.05,0.34


2. For the series in the “hedge fund series” tab, calculate the following statistics related to tail-drawdownrisk.
(a) Skewness
(b) Kurtosis
(c) the fifth quantile of historic returns, which is also known as the Value-at-Risk (VaR)
(d) the mean of the returns at or below the fifth quantile, which is also known as the Conditional
Value-at-Risk (CVaR)
(e) Maximum drawdown - include the dates of the max/min/recovery within the max drawdown period.
There is no need to annualize any of these statistics.

In [35]:
# a) b) c)
hedge_df_summary['Skewness'] = hedge_df.skew()
hedge_df_summary['Kurtosis'] = hedge_df.kurtosis()
hedge_df_summary['VaR'] = hedge_df.quantile(0.05) # 'VaR' is also the 5th quantile
hedge_df_summary['CVaR'] = hedge_df[hedge_df <= hedge_df.quantile(0.05)].mean() # 'CVaR' is also the mean of the returns at or below the 5th quantile

hedge_df_summary

,Mean,Vol,Sharpe,Skewness,Kurtosis,VaR,CVaR
HFRIFWI Index,0.04,0.06,0.69,-0.98,5.96,-0.03,-0.04
MLEIFCTR Index,0.03,0.06,0.54,-0.24,1.69,-0.03,-0.04
MLEIFCTX Index,0.03,0.06,0.51,-0.23,1.66,-0.03,-0.04
HDG US Equity,0.02,0.06,0.33,-0.23,1.80,-0.03,-0.04
QAI US Equity,0.02,0.05,0.34,-0.46,1.83,-0.02,-0.03


In [36]:
hedge_df["QAI US Equity"].index[2]


2

In [37]:
hedge_df_summary.head()

,Mean,Vol,Sharpe,Skewness,Kurtosis,VaR,CVaR
HFRIFWI Index,0.04,0.06,0.69,-0.98,5.96,-0.03,-0.04
MLEIFCTR Index,0.03,0.06,0.54,-0.24,1.69,-0.03,-0.04
MLEIFCTX Index,0.03,0.06,0.51,-0.23,1.66,-0.03,-0.04
HDG US Equity,0.02,0.06,0.33,-0.23,1.80,-0.03,-0.04
QAI US Equity,0.02,0.05,0.34,-0.46,1.83,-0.02,-0.03


In [38]:
# Calculate the maximum drawdown - also the dates of the max/min/recovery within the max drawdown period
cum_returns = (1 + hedge_df).cumprod()
rolling_max = cum_returns.cummax()
drawdown = (cum_returns - rolling_max) / rolling_max
max_drawdown = drawdown.min()
end_date = drawdown.idxmin()
summary = pd.DataFrame({'Max Drawdown': max_drawdown, 'Bottom': end_date})
for col in drawdown:
    summary.loc[col,'Peak'] = (rolling_max.loc[:end_date[col],col]).idxmax()
    recovery = (drawdown.loc[end_date[col]:,col])
    try:
        summary.loc[col,'Recover'] = pd.to_datetime(recovery[recovery >= 0].index[0])
    except:
        summary.loc[col,'Recover'] = pd.to_datetime(None)

In [39]:
summary

,Max Drawdown,Bottom,Peak,Recover
HFRIFWI Index,-0.12,103,100.00,1970-01-01 00:00:00.000000108
MLEIFCTR Index,-0.12,133,118.00,NaT
MLEIFCTX Index,-0.12,133,118.00,NaT
HDG US Equity,-0.14,133,118.00,NaT
QAI US Equity,-0.14,133,118.00,NaT


3. For the series in the “hedge fund series” tab, run a regression of each against SPY (found in the
“merrill factors” tab.) Include an intercept. Report the following regression-based statistics:
(a) Market Beta
(b) Treynor Ratio
(c) Information ratio
No need to annualize the market beta. The Treynor ratio is annualized by multiplying by the
number of periods in a year. The Information ratio is annualized by multiplying by √12

In [40]:
# hedge_df.head()
hedge_df.shape

(142, 5)

In [41]:
SPY = pd.read_excel('proshares_analysis_data.xlsx', sheet_name='merrill_factors', parse_dates=True, index_col=0).loc[:, 'SPY US Equity']
SPY.shape

(142,)

In [43]:
# Merge the two dataframes  - hedge_df and SPY
merged_df = pd.merge(hedge_df, SPY, how='inner', left_index=True, right_index=True)
merged_df.head()

,HFRIFWI Index,MLEIFCTR Index,MLEIFCTX Index,HDG US Equity,QAI US Equity,SPY US Equity


In [44]:
# Define the functions that we need
def performanceMetrics(returns,annualization=1, quantile=.05):
    metrics = pd.DataFrame(index=returns.columns)
    metrics['Mean'] = returns.mean() * annualization
    metrics['Vol'] = returns.std() * np.sqrt(annualization)
    metrics['Sharpe'] = (returns.mean() / returns.std()) * np.sqrt(annualization)

    metrics['Min'] = returns.min()
    metrics['Max'] = returns.max()
    return metrics

def drawdowns(returns):
    cum_returns = (1 + returns).cumprod()
    rolling_max = cum_returns.cummax()
    drawdown = (cum_returns - rolling_max) / rolling_max
    max_drawdown = drawdown.min()
    end_date = drawdown.idxmin()
    summary = pd.DataFrame({'Max Drawdown': max_drawdown, 'Bottom': end_date})
    for col in drawdown:
        summary.loc[col,'Peak'] = (rolling_max.loc[:end_date[col],col]).idxmax()
        recovery = (drawdown.loc[end_date[col]:,col])
        try:
            summary.loc[col,'Recover'] = pd.to_datetime(recovery[recovery >= 0].index[0])
        except:
            summary.loc[col,'Recover'] = pd.to_datetime(None)
    return summary

def regression(returns, benchmark):
    reg = LinearRegression().fit(benchmark.values.reshape(-1,1), returns)
    return reg.coef_[0], reg.intercept_

def regressionMetrics(returns, benchmark, annualization=1):
    beta, alpha = regression(returns, benchmark)
    metrics = pd.DataFrame(index=returns.columns)
    metrics['Market Beta'] = beta
    metrics['Treynor Ratio'] = (returns.mean() - benchmark.mean()) / beta
    metrics['Information Ratio'] = (returns.mean() - benchmark.mean()) / (returns.std() * np.sqrt(annualization))
    return metrics



In [45]:
# Run the regression
regression_df = pd.DataFrame(index=hedge_df.columns)
regression_df['Market Beta'] = np.nan
regression_df['Treynor Ratio'] = np.nan
regression_df['Information Ratio'] = np.nan

In [46]:
# Some useful packages and functions

# Import the packages we need

import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Define the functions that we need

#Mean, Volatality and Sharpe Ratio
def performanceMetrics(returns,annualization=1, quantile=.05):
    metrics = pd.DataFrame(index=returns.columns)
    metrics['Mean'] = returns.mean() * annualization
    metrics['Vol'] = returns.std() * np.sqrt(annualization)
    metrics['Sharpe'] = (returns.mean() / returns.std()) * np.sqrt(annualization)

    metrics['Min'] = returns.min()
    metrics['Max'] = returns.max()
    return metrics


#VaR, CVaR, Max Drawdown
def tailMetrics(returns, quantile=.05, relative=False, mdd=True):
    metrics = pd.DataFrame(index=returns.columns)
    metrics['Skewness'] = returns.skew()
    metrics['Kurtosis'] = returns.kurtosis()

    VaR = returns.quantile(quantile)
    CVaR = (returns[returns < returns.quantile(quantile)]).mean()

    if relative:
        VaR = (VaR - returns.mean())/returns.std()
        CVaR = (CVaR - returns.mean())/returns.std()

    metrics[f'VaR ({quantile})'] = VaR
    metrics[f'CVaR ({quantile})'] = CVaR

    if mdd:
        mdd_stats = maximumDrawdown(returns)
        metrics = metrics.join(mdd_stats)

        if relative:
            metrics['Max Drawdown'] = (metrics['Max Drawdown'] - returns.mean())/returns.std()

    return metrics


def maximumDrawdown(returns):
    cum_returns = (1 + returns).cumprod()
    rolling_max = cum_returns.cummax()
    drawdown = (cum_returns - rolling_max) / rolling_max

    max_drawdown = drawdown.min()
    end_date = drawdown.idxmin()
    summary = pd.DataFrame({'Max Drawdown': max_drawdown, 'Bottom': end_date})

    for col in drawdown:
        summary.loc[col,'Peak'] = (rolling_max.loc[:end_date[col],col]).idxmax()
        recovery = (drawdown.loc[end_date[col]:,col])
        try:
            summary.loc[col,'Recover'] = pd.to_datetime(recovery[recovery >= 0].index[0])
        except:
            summary.loc[col,'Recover'] = pd.to_datetime(None)

        summary['Peak'] = pd.to_datetime(summary['Peak'])
        try:
            summary['Duration (to Recover)'] = (summary['Recover'] - summary['Peak'])
        except:
            summary['Duration (to Recover)'] = None
            
        summary = summary[['Max Drawdown','Peak','Bottom','Recover','Duration (to Recover)']]

    return summary


#Run OLS
def get_ols_metrics(regressors, targets, annualization=1, ignorenan=True):
    # ensure regressors and targets are pandas dataframes, as expected
    if not isinstance(regressors, pd.DataFrame):
        regressors = regressors.to_frame()
    if not isinstance(targets, pd.DataFrame):
        targets = targets.to_frame()

    # align the targets and regressors on the same dates
    df_aligned = targets.join(regressors, how='inner', lsuffix='y ')
    Y = df_aligned[targets.columns]
    Xset = df_aligned[regressors.columns]

    reg = pd.DataFrame(index=targets.columns)
    for col in Y.columns:
        y = Y[col]
        
        if ignorenan:
            # ensure we use only non-NaN dates
            alldata = Xset.join(y,lsuffix='X')
            mask = alldata.notnull().all(axis=1)
            y = y[mask]
            X = Xset[mask]
        else:
            X = Xset

        model = LinearRegression().fit(X, y)
        reg.loc[col, 'alpha'] = model.intercept_ * annualization
        reg.loc[col, regressors.columns] = model.coef_
        reg.loc[col, 'r-squared'] = model.score(X, y)

        # sklearn does not return the residuals, so we need to build them
        yfit = model.predict(X)
        residuals = y - yfit

        # Treynor Ratio is only defined for univariate regression
        if Xset.shape[1] == 1:
            reg.loc[col,'Treynor Ratio'] = (y.mean() / model.coef_) * annualization

        
        # if intercept =0, numerical roundoff will nonetheless show nonzero Info Ratio
        num_roundoff = 1e-12
        if np.abs(model.intercept_) < num_roundoff:
            reg.loc[col, 'Info Ratio'] = None
        else:
            reg.loc[col, 'Info Ratio'] = (model.intercept_ / residuals.std()) * np.sqrt(annualization)

    return reg

In [47]:
# Read the data

#Descriptions sheet. Note the use of .rename to correct column names that are not correctely named in the source file
desc = pd.read_excel('proshares_analysis_data.xlsx','descriptions').rename(columns={'Unnamed: 0':'Symbol'}).set_index('Symbol')

#Hedge Fund Data
hf = pd.read_excel('proshares_analysis_data.xlsx','hedge_fund_series').set_index('date')

#Merrill Lynch Factor Data
mf = pd.read_excel('proshares_analysis_data.xlsx','merrill_factors').set_index('date')

#Other Data
od = pd.read_excel('proshares_analysis_data.xlsx','other_data').set_index('date')

KeyError: "None of ['date'] are in the columns"

### 1. For the series in the “hedge fund series” tab, report the following summary statistics (annualize these statistics):
#### (a) mean
#### (b) volatility
#### (c) Sharpe ratio

In [ ]:
performance_meterics_table = performanceMetrics(hf, annualization=12)
display(performance_meterics_table.style.format\
        (formatter = {'Sharpe' : '{:,.2f}', 'Mean': '{:,.2%}', 'Vol': '{:,.2%}', 'Min': '{:,.2%}', 'Max': '{:,.2%}'}))

### 2. For the series in the “hedge fund series” tab, calculate the following statistics related to tail- risk (no need to annualize any of these statistics).
#### (a) Skewness
#### (b) Excess Kurtosis (in excess of 3)
#### (c) VaR (.05) - the fifth quantile of historic returns
#### (d) CVaR (.05) - the mean of the returns at or below the fifth quantile
#### (e) Maximum drawdown - include the dates of the max/min/recovery within the max drawdown period.

In [ ]:
tail_metrics_table = tailMetrics(hf)
tail_metrics_table['Kurtosis'] = tail_metrics_table['K urtosis'] - 3
display(tail_metrics_table)

### 3. For the series in the “hedge fund series” tab, run a regression of each against SPY (found in the “merrill factors” tab.) Include an intercept. Report the following regression-based statistics (annualize as necessary):
#### (a) Market Beta
#### (b) Treynor Ratio
#### (c) Information ratio

In [ ]:
get_ols_metrics(mf['SPY US Equity'], hf, annualization=12, ignorenan=True)

### 4. Relative Performance
Discuss the previous statistics, and what they tell us about...



#### (a) the differences between SPY and the hedge-fund series?
- Compared to SPY, all hedge fund series have negative alpha (lower mean excess return compared to SPY) and hence negative information ratio (Sharpe of the residual return)
- All series have SPY/market beta near the 0.3-0.4 range, which means they all have a fairly low exposure to the market.
- Given that they have some SPY beta, their total return is positive (Sharpe is positive) but when we strip this factor out, the hedged performance has a negative Sharpe ratio as seen by the Info Ratio (which is the Sharpe Ratio of the hedged position.)



#### (b) which performs better between HDG and QAI.

- HDG has a lower alpha and information ratio, while QAI has a lower beta. 
- However, when beta is adjusted with the beta risk (Treynor ratio), the data shows that HDG has a lower performance.
- Overall, **QAI performs better than HDG**.



#### (c) whether HDG and the ML series capture the most notable properties of HFRI.

- Both HDG and the ML series fail to deliver the same high returns compensated with the high risk of HRFI. 
- The HFRI also shows a very high excess kurtosis, but all of the hedge-fund series has a very small excess kurtosis.





### 5. Report the correlation matrix for these assets.
#### (a) Show the correlations as a heat map.

In [ ]:
cor_table = pd.concat([hf,mf], axis=1).corr()
plt.figure(figsize=(12,10))
sns.heatmap(cor_table, annot=True)
plt.show()

#### (b) Which series have the highest and lowest correlations?

In [ ]:
cor_table = hf.corr()
cor_table[cor_table == 1] = None
cor_table = cor_table.unstack().dropna()
max_cor = cor_table.max()
min_cor = cor_table.min()

print('Strongest correlation is between: '+ str(cor_table[cor_table==max_cor].index[0]))
print('Weakest correlation is between: '+ str(cor_table[cor_table==min_cor].index[0]))

### 6. Replicate HFRI with the six factors listed on the “merrill factors” tab. Include a constant, and run the unrestricted regression,
$$
\begin{align}
r^{hfri}_t &= \alpha^{merr} + x^{merr}_t\beta^{merr} + \epsilon^{merr}_t \\
\hat{r}^{hfri}_t &\equiv \hat{\alpha}^{merr} + x^{merr}_t\hat{\beta}^{merr}
\end{align}
$$
### Note that the second equation is just our notation for the fitted replication.
#### (a) Report the intercept and betas.

In [ ]:
ols_meterics = get_ols_metrics(mf, hf, annualization=12)
names = list(mf.columns)
names.append('alpha')
display(ols_meterics[list(names)])

#### (b) Are the betas realistic position sizes, or do they require huge long-short positions?

The betas shows large positions in USGGM3M index, sometimes it's beta is more than 10 times the beta of SPY. But that's also because USGGM3M has lower volatality, so if we standardize betas by volatality, we see that the beta are similar and so the position sizes are realistic. Below calculation supports this argument

In [ ]:
vol = np.array(performanceMetrics(mf)['Vol'])
beta = np.array(ols_meterics.loc['HFRIFWI Index'][list(mf.columns)])
beta_weights = beta*vol
print(beta_weights)

#### (c) Report the R-squared.

In [ ]:
display(pd.DataFrame(ols_meterics['r-squared']).style.format('{:,.2%}'))

#### (d) Report the volatility of $\epsilon^{merr}$ (the tracking error).

In [ ]:
display(pd.DataFrame(ols_meterics.alpha/ols_meterics['Info Ratio']).style.format('{:,.2%}'))